In [1]:
# !pyenv install 3.12.4
# !pyenv local 3.12.4
# which python3

# !python -m venv .venv
# !source .venv/bin/activate

In [2]:
# !pip install -r requirements.txt

In [3]:
import os
import json
import logging
import requests
from urllib.parse import urlparse
from dotenv import load_dotenv, find_dotenv


In [4]:
load_dotenv(find_dotenv())

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


In [5]:
GITHUB_ACCESS_TOKEN = os.environ.get("GITHUB_ACCESS_TOKEN")


In [6]:
def is_empty(value): 
    if not value or value.strip() == "":
        return True
        
    return False
    

In [7]:
def get_repo_from_url(github_url):
    parsed_url = urlparse(github_url)
    
    path_parts = parsed_url.path.strip('/').split('/')
    
    if len(path_parts) >= 2:
        owner = path_parts[0]
        repo = path_parts[1]
        return owner, repo
    else:
        print("Invalid GitHub URL")
        return None, None
        

In [8]:
def fetch_github_data(endpoint, repo, params=None):
    if is_empty(GITHUB_ACCESS_TOKEN):
        logging.error("GITHUB_ACCESS_TOKEN is not set, is None, or is an empty string.")
        return False
      
    url = f"https://api.github.com/repos/{repo}/{endpoint}"

    headers = {
        # "Authorization": f"Bearer {GITHUB_ACCESS_TOKEN}",
        "Accept": "application/vnd.github+json",
        "X-GitHub-Api-Version": "2022-11-28"
    }

    logging.info(headers)

    logging.info(f"Fetching data from {url} with params: {params}")
    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        logging.error(f"Failed to fetch data: HTTP {response.status_code}")
        logging.error(f"Response text: {response.text}")
        return None
    
    try:
        data = response.json()
        logging.info(f"Data fetched successfully from {url}")
        return data
    except requests.JSONDecodeError as e:
        logging.error("Failed to parse JSON response")
        logging.exception(e)
        return None
        

In [9]:
def fetch_pull_request(repo, pull_request_url):
    pr_number = pull_request_url.split('/')[-1]
    endpoint = f"pulls/{pr_number}"
    return fetch_github_data(endpoint, repo)


In [10]:
github_url = "https://github.com/apache/kafka"
owner, repo = get_repo_from_url(github_url)

print(f"Owner: {owner}\nRepository: {repo}")


Owner: apache
Repository: kafka


In [11]:
issues = fetch_github_data(endpoint="issues", repo=f"{owner}/{repo}", params={"state": "open"})
# print(json.dumps(issues, indent=4))


2024-09-02 12:54:23,056 - INFO - {'Accept': 'application/vnd.github+json', 'X-GitHub-Api-Version': '2022-11-28'}
2024-09-02 12:54:23,056 - INFO - Fetching data from https://api.github.com/repos/apache/kafka/issues with params: {'state': 'open'}
2024-09-02 12:54:23,848 - INFO - Data fetched successfully from https://api.github.com/repos/apache/kafka/issues


In [12]:
commits = fetch_github_data(endpoint="commits", repo=f"{owner}/{repo}")
print(json.dumps(commits, indent=4))


2024-09-02 12:54:23,856 - INFO - {'Accept': 'application/vnd.github+json', 'X-GitHub-Api-Version': '2022-11-28'}
2024-09-02 12:54:23,858 - INFO - Fetching data from https://api.github.com/repos/apache/kafka/commits with params: None
2024-09-02 12:54:24,498 - INFO - Data fetched successfully from https://api.github.com/repos/apache/kafka/commits


[
    {
        "sha": "89418b66aeb3e004e6ef16cd42f2f929d48a6718",
        "node_id": "C_kwDOACG9q9oAKDg5NDE4YjY2YWViM2UwMDRlNmVmMTZjZDQyZjJmOTI5ZDQ4YTY3MTg",
        "commit": {
            "author": {
                "name": "Apoorv Mittal",
                "email": "apoorvmittal10@gmail.com",
                "date": "2024-09-01T23:36:26Z"
            },
            "committer": {
                "name": "GitHub",
                "email": "noreply@github.com",
                "date": "2024-09-01T23:36:26Z"
            },
            "message": "KAFKA-17442: Handled persister errors with write async calls (KIP-932) (#16956)\n\nThe PR makes the persister write RPC async. Also handles the errors from persister as per the review comment here:\r\nAddressing review comment for PR: #16397 (comment)\r\n\r\nReviewers: Andrew Schofield <aschofield@confluent.io>, Abhinav Dixit <adixit@confluent.io>, Jun Rao <junrao@gmail.com>",
            "tree": {
                "sha": "7d940250fea90aae52af2

In [13]:
max_iter_for_test = 2

if issues:
    for issue in issues:
        print(f"Issue #{issue['number']}: {issue['title']}")
        if 'pull_request' in issue:
            pr_data = fetch_pull_request("apache/kafka", issue['pull_request']['url'])
            # print(json.dumps(pr_data, indent=4))
        else:
            print("This issue is not a pull request.")

        max_iter_for_test -= 1
        if max_iter_for_test == 0:
            break


2024-09-02 12:54:24,518 - INFO - {'Accept': 'application/vnd.github+json', 'X-GitHub-Api-Version': '2022-11-28'}
2024-09-02 12:54:24,519 - INFO - Fetching data from https://api.github.com/repos/apache/kafka/pulls/17068 with params: None


Issue #17068: Change exceptions like ControllerMovedException to be retriable


2024-09-02 12:54:25,238 - INFO - Data fetched successfully from https://api.github.com/repos/apache/kafka/pulls/17068
2024-09-02 12:54:25,239 - INFO - {'Accept': 'application/vnd.github+json', 'X-GitHub-Api-Version': '2022-11-28'}
2024-09-02 12:54:25,239 - INFO - Fetching data from https://api.github.com/repos/apache/kafka/pulls/17067 with params: None


Issue #17067: KAFKA-17454: Fix failed transactions_mixed_versions_test.py when running with 3.2


2024-09-02 12:54:25,972 - INFO - Data fetched successfully from https://api.github.com/repos/apache/kafka/pulls/17067


In [14]:
import os
import datetime
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv()) 

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

llm_model = "gpt-3.5-turbo"

load_dotenv(find_dotenv()) 

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

llm_model = "gpt-3.5-turbo"

def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )

    print(response)
    return response.choices[0].message["content"]

# get_completion("What is 1+1?")
